# 📚 Vietnamese Text Summarization với mT5, ViT5

## Tổng quan dự án

**Mục tiêu**: Fine-tune models transformer cho tóm tắt văn bản tiếng Việt  
**Dataset**: VLSP 2021 Summarization Task  
**Models**: PhoBERT (extractive), mT5 (abstractive), ViT5 (abstractive)

---

## 📋 Nội dung

1. **Lý thuyết Text Summarization**
2. **Setup Environment & Load Data**
3. **Data Preprocessing & EDA**
4. **Model Implementation**
   - PhoBERT (Extractive)
   - mT5 (Abstractive)
   - ViT5 (Abstractive)
5. **Training & Fine-tuning**
6. **Evaluation với ROUGE metrics**
7. **Model Comparison & Analysis**

---

## 1️⃣ LÝ THUYẾT TEXT SUMMARIZATION

### A. Hai loại Text Summarization

#### **Extractive Summarization** (PhoBERT)

```
Input: "Việt Nam là một đất nước đông dân. Nền kinh tế phát triển nhanh. 
        Văn hóa đa dạng và phong phú."

Extractive: "Việt Nam là một đất nước đông dân. Nền kinh tế phát triển nhanh."
             ↑ Chọn câu quan trọng từ văn bản gốc
```

**Cách hoạt động:**
- Chấm điểm từng câu trong văn bản (sentence scoring)
- Chọn top-k câu có điểm cao nhất
- Sắp xếp lại theo thứ tự xuất hiện

**Ưu điểm:**
- ✅ Đảm bảo ngữ pháp chính xác (vì dùng câu gốc)
- ✅ Không tạo thông tin sai (no hallucination)
- ✅ Training nhanh, ít resource

**Nhược điểm:**
- ❌ Thiếu tính linh hoạt
- ❌ Có thể không mạch lạc
- ❌ Không thể paraphrase

---

#### **Abstractive Summarization** (mT5, ViT5)

```
Input: "Việt Nam là một đất nước đông dân. Nền kinh tế phát triển nhanh. 
        Văn hóa đa dạng và phong phú."

Abstractive: "Việt Nam có dân số đông với nền kinh tế và văn hóa phát triển."
              ↑ Tạo câu mới, tổng hợp thông tin
```

**Cách hoạt động:**
- Encoder đọc và hiểu văn bản
- Decoder sinh ra văn bản tóm tắt từng token
- Attention mechanism tập trung vào phần quan trọng

**Ưu điểm:**
- ✅ Tóm tắt tự nhiên, mạch lạc
- ✅ Có thể paraphrase và tổng hợp
- ✅ Linh hoạt về độ dài

**Nhược điểm:**
- ❌ Có thể tạo thông tin sai (hallucination)
- ❌ Cần nhiều tài nguyên training
- ❌ Phức tạp hơn

---

### B. Model Architecture

#### **Transformer Encoder-Decoder**

```
Input Text → [Tokenizer] → Input IDs
                                |
                                v
                          [ENCODER]
                           (PhoBERT)
                           (mT5-enc)
                           (ViT5-enc)
                                |
                                v
                       Context Vectors
                                |
                                v
                          [DECODER]
                           (mT5-dec)
                           (ViT5-dec)
                                |
                                v
                       Generated Summary
```

#### **Attention Mechanism**

```python
# Simplified attention calculation
Q = Query vectors    # What decoder wants to know
K = Key vectors      # What encoder has
V = Value vectors    # Actual information

Attention(Q, K, V) = softmax(Q @ K^T / sqrt(d_k)) @ V
                      ↑ Attention weights ↑
```

---

### C. Evaluation Metrics

#### **ROUGE (Recall-Oriented Understudy for Gisting Evaluation)**

```python
Reference: "Việt Nam phát triển kinh tế nhanh"
Generated: "Kinh tế Việt Nam phát triển"

ROUGE-1 (Unigram):
  Overlap = {Việt, Nam, phát, triển, kinh, tế} = 5 words
  Recall = 5/5 = 1.0  ✓
  Precision = 5/5 = 1.0  ✓
  F1 = 2 * (1.0 * 1.0) / (1.0 + 1.0) = 1.0

ROUGE-2 (Bigram):
  Reference bigrams: {(Việt,Nam), (Nam,phát), (phát,triển), (triển,kinh), (kinh,tế)}
  Generated bigrams: {(Kinh,tế), (tế,Việt), (Việt,Nam), (Nam,phát), (phát,triển)}
  Overlap = 3
  F1 ≈ 0.6

ROUGE-L (Longest Common Subsequence):
  LCS = "Việt Nam phát triển" (4 words)
  F1 ≈ 0.8
```

#### **Interpretation**

- **ROUGE-1**: Measures word overlap (content preservation)
- **ROUGE-2**: Measures bigram overlap (fluency)
- **ROUGE-L**: Measures sentence coherence

**Good scores:**
- ROUGE-1: 0.40-0.50 (excellent), 0.30-0.40 (good)
- ROUGE-2: 0.20-0.30 (excellent), 0.15-0.20 (good)
- ROUGE-L: 0.35-0.45 (excellent), 0.25-0.35 (good)

---

## 2️⃣ SETUP ENVIRONMENT

In [ ]:
# Install required packages
!pip install transformers==4.35.0 datasets==2.14.6 -q
!pip install rouge-score==0.1.2 sentencepiece==0.1.99 -q
!pip install accelerate==0.24.1 evaluate==0.4.1 -q

print("✅ All packages installed successfully!")

In [ ]:
# Import libraries
import os
import json
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Transformers
from transformers import (
    AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments, Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)

# Metrics
from rouge_score import rouge_scorer

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 3️⃣ LOAD & EXPLORE DATA

### Dataset Structure

VLSP 2021 Summarization dataset bao gồm:
- **article**: Văn bản tin tức gốc (dài)
- **summary**: Tóm tắt reference (ngắn, do con người viết)

Format có thể là:
- CSV: `train.csv`, `test.csv`
- JSON: `train.json`, `test.json`
- Text files: Folder chứa các file `.txt`

In [ ]:
# Load data - Điều chỉnh path theo dataset của bạn
# Ví dụ: nếu upload dataset vào Kaggle với tên 'vlsp2021-summarization'

data_path = '/kaggle/input/vlsp2021-summarization/train.csv'

# Load
df = pd.read_csv(data_path)

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
df.head()

In [ ]:
# Data exploration
print("="*60)
print("DATA STATISTICS")
print("="*60)

# Check for missing values
print(f"\n📊 Missing values:")
print(df.isnull().sum())

# Length statistics
df['article_len'] = df['article'].str.len()
df['summary_len'] = df['summary'].str.len()
df['compression_ratio'] = df['summary_len'] / df['article_len']

print(f"\n📝 Article statistics:")
print(df['article_len'].describe())

print(f"\n📄 Summary statistics:")
print(df['summary_len'].describe())

print(f"\n🔄 Compression ratio:")
print(f"Mean: {df['compression_ratio'].mean():.2%}")
print(f"Median: {df['compression_ratio'].median():.2%}")

In [ ]:
# Visualize data distribution
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Article length distribution
axes[0,0].hist(df['article_len'], bins=50, color='steelblue', alpha=0.7, edgecolor='black')
axes[0,0].axvline(df['article_len'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df["article_len"].mean():.0f}')
axes[0,0].set_xlabel('Length (characters)', fontsize=12)
axes[0,0].set_ylabel('Frequency', fontsize=12)
axes[0,0].set_title('Article Length Distribution', fontsize=14, fontweight='bold')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# Summary length distribution
axes[0,1].hist(df['summary_len'], bins=50, color='coral', alpha=0.7, edgecolor='black')
axes[0,1].axvline(df['summary_len'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df["summary_len"].mean():.0f}')
axes[0,1].set_xlabel('Length (characters)', fontsize=12)
axes[0,1].set_ylabel('Frequency', fontsize=12)
axes[0,1].set_title('Summary Length Distribution', fontsize=14, fontweight='bold')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# Compression ratio
axes[1,0].hist(df['compression_ratio'], bins=50, color='mediumseagreen', alpha=0.7, edgecolor='black')
axes[1,0].axvline(df['compression_ratio'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df["compression_ratio"].mean():.2%}')
axes[1,0].set_xlabel('Ratio', fontsize=12)
axes[1,0].set_ylabel('Frequency', fontsize=12)
axes[1,0].set_title('Compression Ratio Distribution', fontsize=14, fontweight='bold')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# Scatter plot
axes[1,1].scatter(df['article_len'], df['summary_len'], alpha=0.3, s=10)
axes[1,1].set_xlabel('Article Length', fontsize=12)
axes[1,1].set_ylabel('Summary Length', fontsize=12)
axes[1,1].set_title('Article vs Summary Length', fontsize=14, fontweight='bold')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('data_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

## 4️⃣ DATA PREPROCESSING

In [ ]:
def clean_text(text):
    """Clean Vietnamese text"""
    if not isinstance(text, str):
        return ""
    
    # Remove extra whitespace
    text = ' '.join(text.split())
    text = text.strip()
    
    return text

# Clean data
print("Cleaning data...")
df['article'] = df['article'].apply(clean_text)
df['summary'] = df['summary'].apply(clean_text)

# Remove invalid samples
df = df[df['article'].str.len() > 50]
df = df[df['summary'].str.len() > 10]
df = df[df['article'].str.len() < 5000]

print(f"✓ Clean dataset: {len(df)} samples")
print(f"✓ Removed {len(pd.read_csv(data_path)) - len(df)} invalid samples")

In [ ]:
# Split dataset
from sklearn.model_selection import train_test_split

# Train/val/test split: 70/15/15
train_val, test = train_test_split(df, test_size=0.15, random_state=42)
train, val = train_test_split(train_val, test_size=0.15/(1-0.15), random_state=42)

print(f"\n📊 Dataset split:")
print(f"Train: {len(train):,} samples ({len(train)/len(df):.1%})")
print(f"Val: {len(val):,} samples ({len(val)/len(df):.1%})")
print(f"Test: {len(test):,} samples ({len(test)/len(df):.1%})")

# Reset index
train = train.reset_index(drop=True)
val = val.reset_index(drop=True)
test = test.reset_index(drop=True)

## 5️⃣ DATASET & DATALOADER

In [ ]:
class SummarizationDataset(Dataset):
    """Dataset for text summarization"""
    
    def __init__(self, articles, summaries, tokenizer, max_length=512, max_target_length=128):
        self.articles = articles
        self.summaries = summaries
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.max_target_length = max_target_length
    
    def __len__(self):
        return len(self.articles)
    
    def __getitem__(self, idx):
        article = "summarize: " + str(self.articles.iloc[idx])
        summary = str(self.summaries.iloc[idx])
        
        # Tokenize input
        inputs = self.tokenizer(
            article,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        # Tokenize target
        targets = self.tokenizer(
            summary,
            max_length=self.max_target_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': targets['input_ids'].squeeze()
        }

print("✓ Dataset class defined")

## 6️⃣ MODEL TRAINING - ViT5

### ViT5: Vietnamese T5

**ViT5 là model T5 được VietAI pre-train trên corpus tiếng Việt lớn:**
- 12 layers encoder + 12 layers decoder
- 768 hidden dimensions
- Optimized cho tiếng Việt
- Performance tốt hơn mT5 cho Vietnamese tasks

In [ ]:
# Configuration
MODEL_NAME = 'VietAI/vit5-base'
BATCH_SIZE = 4  # Giảm nếu out of memory
LEARNING_RATE = 5e-5
NUM_EPOCHS = 3
MAX_LENGTH = 512
MAX_TARGET_LENGTH = 128

print(f"Model: {MODEL_NAME}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Learning rate: {LEARNING_RATE}")
print(f"Epochs: {NUM_EPOCHS}")

In [ ]:
# Load model và tokenizer
print("Loading ViT5 model...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

print(f"✓ Model loaded on {device}")
print(f"✓ Model parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# Create datasets
train_dataset = SummarizationDataset(
    train['article'],
    train['summary'],
    tokenizer,
    max_length=MAX_LENGTH,
    max_target_length=MAX_TARGET_LENGTH
)

val_dataset = SummarizationDataset(
    val['article'],
    val['summary'],
    tokenizer,
    max_length=MAX_LENGTH,
    max_target_length=MAX_TARGET_LENGTH
)

print(f"✓ Train dataset: {len(train_dataset)} samples")
print(f"✓ Val dataset: {len(val_dataset)} samples")

In [ ]:
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    padding=True
)

print("✓ Data collator created")

In [ ]:
# Metric computation
rouge_scorer_obj = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    
    # Decode predictions
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Compute ROUGE
    rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
    
    for pred, label in zip(decoded_preds, decoded_labels):
        scores = rouge_scorer_obj.score(label, pred)
        rouge_scores['rouge1'].append(scores['rouge1'].fmeasure)
        rouge_scores['rouge2'].append(scores['rouge2'].fmeasure)
        rouge_scores['rougeL'].append(scores['rougeL'].fmeasure)
    
    return {
        'rouge1': np.mean(rouge_scores['rouge1']),
        'rouge2': np.mean(rouge_scores['rouge2']),
        'rougeL': np.mean(rouge_scores['rougeL'])
    }

print("✓ Metric function defined")

In [ ]:
# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir='./vit5_summarization',
    evaluation_strategy='steps',
    eval_steps=500,
    logging_steps=100,
    save_steps=500,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=NUM_EPOCHS,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
    gradient_accumulation_steps=2,
    warmup_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model='rouge1',
    greater_is_better=True,
    report_to='none'
)

print("✓ Training arguments configured")

In [ ]:
# Initialize Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

print("✓ Trainer initialized")
print("\n🚀 Ready to train!")

In [ ]:
# Start training
print("\n" + "="*60)
print("TRAINING ViT5 MODEL")
print("="*60 + "\n")

train_result = trainer.train()

print("\n✅ Training completed!")
print(f"Final training loss: {train_result.metrics['train_loss']:.4f}")

In [ ]:
# Save model
trainer.save_model('./vit5_final')
tokenizer.save_pretrained('./vit5_final')

print("✓ Model saved to ./vit5_final")

## 7️⃣ EVALUATION

In [ ]:
# Evaluate on test set
print("\n" + "="*60)
print("EVALUATING ON TEST SET")
print("="*60 + "\n")

model.eval()

results = {
    'rouge1': [],
    'rouge2': [],
    'rougeL': [],
    'predictions': [],
    'references': []
}

with torch.no_grad():
    for idx in tqdm(range(len(test)), desc="Evaluating"):
        article = "summarize: " + str(test.iloc[idx]['article'])
        reference = str(test.iloc[idx]['summary'])
        
        # Generate
        inputs = tokenizer(
            article,
            max_length=MAX_LENGTH,
            truncation=True,
            return_tensors='pt'
        ).to(device)
        
        outputs = model.generate(
            **inputs,
            max_length=MAX_TARGET_LENGTH,
            num_beams=4,
            length_penalty=0.6,
            early_stopping=True
        )
        
        prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Compute ROUGE
        scores = rouge_scorer_obj.score(reference, prediction)
        results['rouge1'].append(scores['rouge1'].fmeasure)
        results['rouge2'].append(scores['rouge2'].fmeasure)
        results['rougeL'].append(scores['rougeL'].fmeasure)
        results['predictions'].append(prediction)
        results['references'].append(reference)

# Final results
print(f"\n📊 Test Results:")
print(f"ROUGE-1: {np.mean(results['rouge1']):.4f} ± {np.std(results['rouge1']):.4f}")
print(f"ROUGE-2: {np.mean(results['rouge2']):.4f} ± {np.std(results['rouge2']):.4f}")
print(f"ROUGE-L: {np.mean(results['rougeL']):.4f} ± {np.std(results['rougeL']):.4f}")

## 8️⃣ ANALYSIS & VISUALIZATION

In [ ]:
# Plot ROUGE distribution
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

scores = ['rouge1', 'rouge2', 'rougeL']
titles = ['ROUGE-1', 'ROUGE-2', 'ROUGE-L']
colors = ['steelblue', 'coral', 'mediumseagreen']

for idx, (score, title, color) in enumerate(zip(scores, titles, colors)):
    axes[idx].hist(results[score], bins=30, alpha=0.7, color=color, edgecolor='black')
    axes[idx].axvline(np.mean(results[score]), color='red', linestyle='--', 
                    linewidth=2, label=f'Mean: {np.mean(results[score]):.3f}')
    axes[idx].set_xlabel('Score', fontsize=12)
    axes[idx].set_ylabel('Frequency', fontsize=12)
    axes[idx].set_title(f'{title} Distribution', fontsize=14, fontweight='bold')
    axes[idx].legend(fontsize=11)
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('rouge_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Sample predictions
print("\n" + "="*80)
print("SAMPLE PREDICTIONS")
print("="*80 + "\n")

n_samples = 5
indices = np.random.choice(len(results['predictions']), n_samples, replace=False)

for i, idx in enumerate(indices):
    print(f"\n{'─'*80}")
    print(f"Sample {i+1}")
    print(f"{'─'*80}")
    print(f"\n📝 Reference:")
    print(f"{results['references'][idx]}")
    print(f"\n🤖 Generated:")
    print(f"{results['predictions'][idx]}")
    print(f"\n📊 Scores:")
    print(f"ROUGE-1: {results['rouge1'][idx]:.3f} | "
          f"ROUGE-2: {results['rouge2'][idx]:.3f} | "
          f"ROUGE-L: {results['rougeL'][idx]:.3f}")

## 9️⃣ SAVE RESULTS

In [ ]:
# Save detailed results
results_df = pd.DataFrame({
    'reference': results['references'],
    'prediction': results['predictions'],
    'rouge1': results['rouge1'],
    'rouge2': results['rouge2'],
    'rougeL': results['rougeL']
})

results_df.to_csv('test_results.csv', index=False)
print("✓ Results saved to test_results.csv")

# Save summary statistics
summary = {
    'model': MODEL_NAME,
    'rouge1_mean': float(np.mean(results['rouge1'])),
    'rouge1_std': float(np.std(results['rouge1'])),
    'rouge2_mean': float(np.mean(results['rouge2'])),
    'rouge2_std': float(np.std(results['rouge2'])),
    'rougeL_mean': float(np.mean(results['rougeL'])),
    'rougeL_std': float(np.std(results['rougeL'])),
    'test_samples': len(test)
}

with open('summary_statistics.json', 'w', encoding='utf-8') as f:
    json.dump(summary, f, indent=2, ensure_ascii=False)

print("✓ Summary saved to summary_statistics.json")

print("\n✅ ALL DONE!")